In [ ]:
#weights = [0.1 1 1 1 1]
# gpu 5
# cell of checkpoints not included

In [ ]:
"""

clearly a problem in the database, the minimum number of the image is -0.1 (?)

good paper about losses:
https://arxiv.org/pdf/2006.14822.pdf

transfer learning in encoder with tumor relared neural network
how to implement dropout
number of parameters in network
resnet34 or densenet121

TODOs:

check the activation of layers --> https://awjuliani.medium.com/visualizing-neural-network-layer-activation-tensorflow-tutorial-d45f8bf7bbc4
implement shuffling in the generator --> https://github.com/qubvel/segmentation_models/blob/master/examples/multiclass%20segmentation%20(camvid).ipynb // https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly 
"""

In [ ]:
# from tensorflow.keras import layers
# from tensorflow import keras
# import segmentation_models as sm

# def get_model_with_dropout(base_model_name='efficientnetb4',activation='sigmoid',dropout = 0.1):
#     base_model = sm.Unet(base_model_name, encoder_weights='imagenet')
#     base_model_input = base_model.input
#     base_model_output = base_model.get_layer('final_conv').output
#     #add dropout
#     base_model_output = keras.layers.Dropout(dropout)(base_model_output)
#     #add activation
#     output = keras.layers.Activation(activation, name=activation)(base_model_output)
#     model_dp = keras.models.Model(base_model_input, output)

#     return model_dp

In [ ]:
import os
from tensorflow.python.client import device_lib
import tensorflow as tf

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

#Run from server

gpu4 = "GPU-71b4cdfc-e381-0b98-9b24-4fc06284b496" 
gpu5 = "GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1" 
gpu6 = "GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f"
gpu7 = "GPU-c0a8738f-6dd0-1b78-c38f-4969fd3886a8"

os.environ["CUDA_VISIBLE_DEVICES"]= gpu7# + "," + gpu6
print(get_available_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import h5py
import cv2
import pickle

In [ ]:

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import History

In [ ]:
IMG_HEIGHT = 240
IMG_WIDTH = 240
IMG_DEPTH = 155

IMG_HEIGHT_UNET = 256
IMG_WIDTH_UNET = 256

N_IMG = 369
length_file = IMG_DEPTH * N_IMG
h5py_file_name = 'brats2020_normalized.hdf5'
smooth = 1e-4

___

Generating the data and creating data arrays

In [ ]:
ImgDir = os.path.join("..","data","MICCAI_BraTS2020_TrainingData")
features_path = list()
labels_path = list()
count = 0
lim = 10

for folder in os.listdir(ImgDir):
    count +=1
    if 'Training' in folder:
        new_dir = os.path.join(ImgDir,folder)
        data = os.listdir(new_dir)
        for files in data:
            if 'flair' in files:
                features_path.append(os.path.join(new_dir, files))
            if 'seg' in files:
                labels_path.append(os.path.join(new_dir, files))

print(len(features_path))
print(len(labels_path))


In [ ]:
img_conc_features = np.zeros((len(features_path),IMG_HEIGHT,IMG_WIDTH,IMG_DEPTH))
path = features_path

for file, i in zip(path,range(len(path))):
    img = nib.load(file)
    imgarr = img.get_fdata()
    img_conc_features[i,:,:,:] = imgarr
    
img_conc_features = np.concatenate(img_conc_features,axis=2)
print(np.shape(img_conc_features))

In [ ]:
img_conc_labels = np.zeros((len(labels_path),IMG_HEIGHT,IMG_WIDTH,IMG_DEPTH))
path = labels_path

for file, i in zip(path,range(len(path))):
    img = nib.load(file)
    imgarr = img.get_fdata()
    img_conc_labels[i,:,:,:] = imgarr
    
img_conc_labels = np.concatenate(img_conc_labels,axis=2)
print(np.shape(img_conc_labels))

In [ ]:
with h5py.File(os.path.join('..','data',h5py_file_name), 'a') as f:
    f.create_dataset("features", data=img_conc_features, compression="gzip")
    f.create_dataset("labels", data=img_conc_labels, compression="gzip")

In [ ]:
hdf5_store_Brats = h5py.File(os.path.join('..','data',h5py_file_name), "r")
test_img = hdf5_store_Brats["features"][:, :, 100]
plt.imshow(test_img, cmap ='gray')
hdf5_store_Brats.close()

In [ ]:
hdf5_store_Brats = h5py.File(os.path.join('..','data',h5py_file_name), "r")
test_img = hdf5_store_Brats["labels"][:, :, 100]
plt.imshow(test_img, cmap ='gray')
hdf5_store_Brats.close()

___

Load data and model

In [ ]:
train_begin = 0
train_stop = 20000
n_images = 21000

# n_images = 57195
# train_stop = int(np.floor(n_images * 0.9))

#https://www.pythonforthelab.com/blog/how-to-use-hdf5-files-in-python/  

# with h5py.File(os.path.join('..','data',h5py_file_name), "r") as f:
#     #images_train = f["features"][()] #the whole dataset: 57195 images
#     #labels_train = f["labels"][()]

#     images = f["features"]
#     images_train = tf.constant(images[:,:,train_begin:train_stop],dtype=tf.float32) #taking a subsample
#     images_val = tf.constant(images[:,:,train_stop:n_images+1],dtype=tf.float32)

#     labels = f["labels"]
#     labels_train = tf.constant(labels[:,:,train_begin:train_stop],dtype=tf.int8)
#     labels_val = tf.constant(labels[:,:,train_stop:n_images+1],dtype=tf.int8)

# print(images_val[:,:,1])

In [ ]:
with h5py.File(os.path.join('..','data',h5py_file_name), "r") as f:
    #images_train = f["features"][()] #the whole dataset: 57195 images
    #labels_train = f["labels"][()]

    images = f["features"]
    images_train = images[:,:,train_begin:train_stop]
    images_val = images[:,:,train_stop:n_images+1]

    labels = f["labels"]
    labels_train = labels[:,:,train_begin:train_stop]
    labels_val = labels[:,:,train_stop:n_images+1]

print(images_val[:,:,1])

In [ ]:
images_train.get_shape().as_list()[2]

In [ ]:
plt.imshow(images_train[:,:,100], interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
plt.imshow(labels_train[:,:,100], interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
images_val[:,:,0]

____

In [ ]:
CLASSES = ['0','1','2','3','4']
LR = 1e-4
EPOCHS = 50
n_classes = 5
BATCH_SIZE = 32

In [ ]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x

def visualize_histories(**images):
    """Import as tuples: one = (a,b)"""
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.ylim(0,1)
        plt.xlabel('epoch')
        if i == 0:
            plt.ylabel('dice_score')
        plt.title(' '.join(name.split('_')).title())
        plt.plot(np.asarray(image[0]))
        plt.plot(np.asarray(image[1]))
        plt.legend(['train', 'val'])
    plt.show()

In [ ]:
class Dataset:
    
    def __init__(self, images_train, images_label, classes = CLASSES):
        self.images_fps = images_train
        self.masks_fps = images_label
        self.classes = classes
        self.class_values = [self.classes.index(cls.lower()) for cls in self.classes]
        self.ids = np.shape(self.images_fps)[2]
        #self.ids = images_train.get_shape().as_list()[2]
    
    def __getitem__(self, i):
        
        X_new = np.zeros((IMG_HEIGHT_UNET, IMG_WIDTH_UNET, 3))
        y_new = np.zeros((IMG_HEIGHT_UNET, IMG_WIDTH_UNET))
        
        X_new[:IMG_HEIGHT,:IMG_WIDTH,0] = self.images_fps[:,:,i]
        X_new[:IMG_HEIGHT,:IMG_WIDTH,1] = self.images_fps[:,:,i]
        X_new[:IMG_HEIGHT,:IMG_WIDTH,2] = self.images_fps[:,:,i]

        #image = X_new[:,:,:] / np.max(X_new[:,:,:])
        #image = X_new[:,:,:]
        
        y_new[:IMG_HEIGHT,:IMG_WIDTH] = self.masks_fps[:,:,i]
        
        #mask = y_new
        
        # extract certain classes from mask (e.g. cars)
        #masks = [(mask == v) for v in self.class_values]
        #mask = np.stack(masks, axis=-1)
        
        #mask = np.stack(masks, axis=-1).astype('float')
        
        # add background if mask is not binary
        #if mask.shape[-1] != 1:
            #background = 1 - mask.sum(axis=-1, keepdims=True)
            #mask = np.concatenate((mask, background), axis=-1)
        
        return X_new, tf.keras.utils.to_categorical(y_new, num_classes=n_classes)
    
    def __len__(self):
        return self.ids

In [ ]:
class Dataloder(tf.keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=BATCH_SIZE, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        #self.indexes = np.arange(len(dataset))
        self.on_epoch_end()

    def __getitem__(self, index):
        
        # collect batch data
        #print("self indexes: ",self.indexes)
        if "data" in dir():
            del data
        if "batch" in dir():
            del batch
        
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [np.arange(len(self.dataset))[k] for k in indexes]

        #print("temporal IDs: ", list_IDs_temp)
        #start = i * self.batch_size
        #stop = (i + 1) * self.batch_size
        data = []
        batch = []
        
        for j in list_IDs_temp:
            # data.append(self.dataset[self.indexes[j]])
            data.append(self.dataset[j])
        
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]

        
        return (batch[0],batch[1])
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.indexes) / self.batch_size))
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        self.indexes = np.arange(len(self.dataset))
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)  

In [ ]:
#calculates dice considering an input with a single class
def dice_single(true,pred):
    true = K.batch_flatten(true)
    pred = K.batch_flatten(pred)
    pred = K.round(pred)

    intersection = K.sum(true * pred, axis=-1)
    true = K.sum(true, axis=-1)
    pred = K.sum(pred, axis=-1)

    return ((2*intersection) + K.epsilon()) / (true + pred + K.epsilon())

def dice_inner_0(true,pred,index=0):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_1(true,pred,index=1):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_2(true,pred,index=2):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_3(true,pred,index=3):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_4(true,pred,index=4):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

# metrics_test5 = [dice_inner_0, dice_inner_1, dice_inner_2, dice_inner_3, dice_inner_4, sm.metrics.FScore(threshold=0.5)]
metrics_test5 = [dice_inner_0, dice_inner_1, dice_inner_2, dice_inner_3, dice_inner_4]

In [ ]:
def weighted_categorical_crossentropy(weights):
    def wcce(y_true, y_pred):
        Kweights = K.constant(weights)
        if not tf.is_tensor(y_pred): y_pred = K.constant(y_pred)
        y_true = K.cast(y_true, y_pred.dtype)
        return K.categorical_crossentropy(y_true, y_pred) * K.sum(y_true * Kweights, axis=-1)
    return wcce

In [ ]:
def dice_single(true,pred):
    true = K.batch_flatten(true)
    pred = K.batch_flatten(pred)
    pred = K.round(pred)

    intersection = K.sum(true * pred, axis=-1)
    true = K.sum(true, axis=-1)
    pred = K.sum(pred, axis=-1)

    return ((2*intersection) + K.epsilon()) / (true + pred + K.epsilon())

def dice_inner_0(true,pred,index=0):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_1(true,pred,index=1):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_2(true,pred,index=2):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_3(true,pred,index=3):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_4(true,pred,index=4):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

In [ ]:
def average(x, per_image=False, class_weights=None, **kwargs):
    if per_image:
        x = K.mean(x, axis=0)
    if class_weights is not None:
        x = x * class_weights
    return K.mean(x)

def my_func(arg):
    arg = tf.convert_to_tensor(arg, dtype=tf.float32)
    return arg

def dice_loss_weighted(y_true, y_pred):
    
    arg = my_func([dice_inner_0(y_true, y_pred),dice_inner_1(y_true, y_pred),dice_inner_2(y_true, y_pred),dice_inner_3(y_true, y_pred),dice_inner_4(y_true, y_pred)])
    score = average(arg)
    return  1-score

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_multilabel_loss(y_true, y_pred, numLabels=5):
    dice=0
    for index in range(numLabels):
        dice -= dice_coef(y_true[:,:,:,index], y_pred[:,:,:,index])
    print(dice)
    return 1+dice



In [ ]:
def dice_coef_multilabel_loss(y_true, y_pred, numLabels=3, class_weights = [0,1,2]):
    dice=0
    #sth = tf.Variable(np.empty((), dtype='float32'))
    #sth = []
    for index in range(numLabels):
        dice = tf.unstack(dice_coef(y_true[:,:,:,index], y_pred[:,:,:,index]))
        #sth = tf.concat([sth,dice],axis=0)
        #sth.append(dice)
    weighted_dice = K.constant(sth) * K.constant(class_weights)
    return K.constant(1 - np.mean(weighted_dice))

In [ ]:
weights = [0.01,2,1,0.01,1]

In [ ]:
def f_score_loss(gt, pr, beta=1, class_weights=weights, class_indexes=None, smooth=smooth, per_image=False, threshold=None, **kwargs):
    gt, pr = gather_channels(gt, pr, indexes=class_indexes, **kwargs)
    pr = round_if_needed(pr, threshold, **kwargs)
    axes = get_reduce_axes(per_image, **kwargs)

    # calculate score
    tp = K.sum(gt * pr, axis=axes)
    fp = K.sum(pr, axis=axes) - tp
    fn = K.sum(gt, axis=axes) - tp

    score = ((1 + beta ** 2) * tp + smooth) \
            / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + smooth)
    score = average(score, per_image, class_weights, **kwargs)

    return 1-score

In [ ]:
def _gather_channels(x, indexes, **kwargs):
    """Slice tensor along channels axis by given indexes"""
    backend = K
    if backend.image_data_format() == 'channels_last':
        x = backend.permute_dimensions(x, (3, 0, 1, 2))
        x = backend.gather(x, indexes)
        x = backend.permute_dimensions(x, (1, 2, 3, 0))
    else:
        x = backend.permute_dimensions(x, (1, 0, 2, 3))
        x = backend.gather(x, indexes)
        x = backend.permute_dimensions(x, (1, 0, 2, 3))
    return x

def get_reduce_axes(per_image, **kwargs):
    backend = K
    axes = [1, 2] if backend.image_data_format() == 'channels_last' else [2, 3]
    if not per_image:
        axes.insert(0, 0)
    return axes

def gather_channels(*xs, indexes=None, **kwargs):
    """Slice tensors along channels axis by given indexes"""
    if indexes is None:
        return xs
    elif isinstance(indexes, (int)):
        indexes = [indexes]
    xs = [_gather_channels(x, indexes=indexes, **kwargs) for x in xs]
    return xs

def average(x, per_image=False, class_weights=None, **kwargs):
    backend = K
    if per_image:
        x = backend.mean(x, axis=0)
    if class_weights is not None:
        x = x * class_weights
    return backend.mean(x)

def round_if_needed(x, threshold, **kwargs):
    backend = K
    if threshold is not None:
        x = backend.greater(x, threshold)
        x = backend.cast(x, backend.floatx())
    return x

def iou_score_loss(gt, pr, class_weights=[0.1,1,1,1,1], class_indexes=None, smooth=smooth, per_image=False, threshold=None, **kwargs):
    backend = K
    gt, pr = gather_channels(gt, pr, indexes=class_indexes, **kwargs)
    pr = round_if_needed(pr, threshold, **kwargs)
    axes = get_reduce_axes(per_image, **kwargs)

    # score calculation
    intersection = backend.sum(gt * pr, axis=axes)
    union = backend.sum(gt + pr, axis=axes) - intersection

    score = (intersection + smooth) / (union + smooth)
    print(score)
    score = average(score, per_image, class_weights, **kwargs)

    return 1-score


In [ ]:

#loss = weighted_categorical_crossentropy(weights)
loss = f_score_loss
optim = tf.keras.optimizers.Adam(LR)

In [ ]:
# def get_model():
#     in1 = Input(shape=(IMG_HEIGHT_UNET, IMG_WIDTH_UNET, 3))

#     conv1 = Conv2D(4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
#     conv1 = Dropout(0.5)(conv1)
#     conv1 = Conv2D(4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
#     pool1 = MaxPooling2D((2, 2))(conv1)

#     conv2 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
#     conv2 = Dropout(0.5)(conv2)
#     conv2 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
#     pool2 = MaxPooling2D((2, 2))(conv2)

#     conv3 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
#     conv3 = Dropout(0.5)(conv3)
#     conv3 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
#     pool3 = MaxPooling2D((2, 2))(conv3)

#     conv4 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
#     conv4 = Dropout(0.5)(conv4)
#     conv4 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

#     up1 = concatenate([UpSampling2D((2, 2))(conv4), conv3], axis=-1)
#     conv5 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
#     conv5 = Dropout(0.5)(conv5)
#     conv5 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    
#     up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
#     conv6 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
#     conv6 = Dropout(0.5)(conv6)
#     conv6 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

#     up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
#     conv7 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
#     conv7 = Dropout(0.5)(conv7)
#     conv7 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
#     segmentation = Conv2D(5, (1, 1), activation='sigmoid', name='seg')(conv7)

#     model = Model(inputs=[in1], outputs=[segmentation])
    
#     model.compile(optim, loss, metrics_test5)
    
#     return model

In [ ]:
def unet(pretrained_weights = None,input_size = (256,256,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    # drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    # drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(16, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(5, 1, activation = 'softmax')(conv9)

    model = Model(inputs = [inputs], outputs = [conv10])
     #model = Model(inputs=[in1], outputs=[segmentation])

    model.compile(optim, loss=loss, metrics = metrics_test5)
    
    #model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [ ]:
model = unet()
model.summary()

In [ ]:
# tf.debugging.set_log_device_placement(True)
# with tf.device('/CPU:0'):

train_dataset = Dataset(images_train, labels_train, classes=CLASSES)
valid_dataset = Dataset(images_val, labels_val, classes=CLASSES)

train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

# check shapes for errors
assert train_dataloader[0][0].shape == (BATCH_SIZE, IMG_HEIGHT_UNET, IMG_WIDTH_UNET, 3)
assert train_dataloader[0][1].shape == (BATCH_SIZE, IMG_HEIGHT_UNET, IMG_WIDTH_UNET, n_classes)

# define callbacks for learning rate scheduling and best checkpoints saving
# callbacks = [
#     tf.keras.callbacks.ModelCheckpoint('./best_model.h5', save_weights_only=True, save_best_only=True, mode='min'),
#     tf.keras.callbacks.ReduceLROnPlateau(),
# ]

# callbacks = [
#     tf.keras.callbacks.ReduceLROnPlateau(),
# ]

In [ ]:
class printbatch(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, epoch, logs={}):

        res_dir = os.path.join("..","data","test_images")

        try:
            os.makedirs(res_dir)
        except:
            print(f"{res_dir} directory already exist")

    
#     def on_train_batch_begin(self, batch, logs=None):
#         keys = list(logs.keys())
#         print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

#     def on_train_batch_end(self, batch, logs=None):
#         keys = list(logs.keys())
#         print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        
#         res_dir = os.path.join("..","data","test_images")
    
#         x_img = os.path.join(res_dir,"X_input.jpg")
#         y_img = os.path.join(res_dir,"Y_truth.jpg")
#         predicted_img = os.path.join(res_dir,f"{epoch}_Y_predicted.jpg")
        
#         image, gt_mask = train_dataset[100]
#         #image = np.expand_dims(image, axis=0)
#         pr_mask = self.model.predict(image[np.newaxis, ...])
    
#         gt_mask_vis = gt_mask[...,0:3]
#         pr_mask_vis = pr_mask[...,0:3]

# #         visualize(
# #             image=denormalize(image.squeeze()),
# #             gt_mask_vis=gt_mask_vis.squeeze(),
# #             pr_mask_vis=pr_mask_vis.squeeze(),
# #         )

#         cv2.imwrite(x_img, image[:,:,0])
#         cv2.imwrite(y_img, gt_mask_vis.squeeze())
#         cv2.imwrite(predicted_img, pr_mask_vis.squeeze())
#         cv2.imwrite(predicted_img, prediction[0,:,:,:] * 255.)
        
        
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))
        
# pb = printbatch()        

In [ ]:
checkpoint_path =  os.path.join("..","data","checkpoints","training_0211/cp-{epoch:04d}.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)

STEPS_PER_EPOCH = train_stop / BATCH_SIZE
SAVE_PERIOD = 5

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    save_weights_only=True,
    save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH)
    )

# history_callback = tf.keras.callbacks.CSVLogger('history.csv',separator=',',append=False)

# callbacks = [pb, cp_callback]


In [ ]:
image, mask = valid_dataset[100]
visualize(
    image=image[:,:,0], ### if putting the whole image, black and white, could it be the reason of the problem??
    no_tumor=mask[..., 0].squeeze(),
    one=mask[..., 1].squeeze(),
    two=mask[..., 2].squeeze(),
    three=mask[..., 3].squeeze(),
    four=mask[..., 4].squeeze(),
)

In [ ]:
np.shape(mask[...].squeeze())

In [ ]:
history = model.fit(
    train_dataloader,
    epochs=EPOCHS,
    verbose = 1,
    validation_data=valid_dataloader, 
)

In [ ]:
tf.trainable_variables()

In [ ]:
"""
Solutions to RAM memory problem

    def __del__(self):
        #put termination message
        for i in range(self._workers)
            self._notificationQueue.put("terminate")
        #empty queues, otherwise the workers won't terminate
        for i in range(self._max_queue_size):
            self._batchQueue.get(timeout=10)
            

"""

___

### Check for activation of weights

In [ ]:
# https://awjuliani.medium.com/visualizing-neural-network-layer-activation-tensorflow-tutorial-d45f8bf7bbc4


def getActivations(layer,stimuli):
    units = sess.run(layer,feed_dict={x:np.reshape(stimuli,[1,784],order='F'),keep_prob:1.0})
    plotNNFilter(units)
    
    
def plotNNFilter(units):
    filters = units.shape[3]
    plt.figure(1, figsize=(20,20))
    n_columns = 6
    n_rows = math.ceil(filters / n_columns) + 1
    for i in range(filters):
        plt.subplot(n_rows, n_columns, i+1)
        plt.title('Filter ' + str(i))
        plt.imshow(units[0,:,:,i], interpolation="nearest", cmap="gray")
        

In [ ]:
model.layers[0]

In [ ]:
imageToUse = image[:,:,0]
plt.imshow(np.reshape(imageToUse,[256,256]), interpolation="nearest", cmap="gray")

In [ ]:
image

### Visualizing results

In [ ]:
visualize_histories(
    class_zero = (history.history['dice_inner_0'], history.history['val_dice_inner_0']), ### if putting the whole image, black and white, could it be the reason of the problem??
    class_one = (history.history['dice_inner_1'], history.history['val_dice_inner_1']),
    class_two = (history.history['dice_inner_2'], history.history['val_dice_inner_2']),
    class_three = (history.history['dice_inner_3'], history.history['val_dice_inner_3']),
    class_four = (history.history['dice_inner_4'], history.history['val_dice_inner_4'])
)

In [ ]:
checkpoint_path= os.path.join("..","data","checkpoints","unet_brats.ckpt")

In [ ]:
# model.save_weights(checkpoint_path)
# Restore the weights
checkpoint_path = os.path.join('..', 'data', 'checkpoints')
checkpoint_number = 'cp-0040.ckpt'
checkpoint_path = os.path.join(checkpoint_path, checkpoint_number)
model.load_weights(checkpoint_path)

In [ ]:
image, gt_mask = train_dataset[100]
model.evaluate(image[np.newaxis, ...], gt_mask[np.newaxis, ...], verbose=2)

In [ ]:
# Plot training & validation iou_score values
# plt.figure(figsize=(30, 5))
# plt.subplot(121)
# plt.plot(history.history['f1-score'])
# plt.plot(history.history['val_f1-score'])
# plt.title('Model f1-score')
# plt.ylabel('f1-score')
# plt.xlabel('Epoch')
# plt.ylim([0,1])
# plt.legend(['Train', 'Val'], loc='upper left')

# Plot training & validation loss values

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.ylim([0,1])
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
n = 1
# ids = np.random.choice(np.arange(len(labels_train)), size=n)
ids = [80]
for i in ids:
    
    image, gt_mask = valid_dataset[i]
    #image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image[np.newaxis, ...])
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[...,4].squeeze(),
        pr_mask=pr_mask[...,4].squeeze(),
    )


In [ ]:
colour_mappings[i]

In [ ]:
for i in range(5):
    plt.imshow(pr_mask[0,:,:,1], cmap = colour_mappings[i],alpha = 0.2)

In [ ]:
plt.imshow(pr_mask[0,:,:,4])

Image visualization of training

In [ ]:
test_img_feat_dir = os.path.join("..","data","MICCAI_BraTS2020_TrainingData","BraTS20_Training_001","BraTS20_Training_001_flair.nii.gz")
img_feat = nib.load(test_img_feat_dir)
imgarr_feat = img_feat.get_fdata()
test_img_feat_slice = imgarr_feat[:,:,100]
test_img = np.zeros((240, 240 ,3), np.float32)

test_img[:,:,0] = test_img_feat_slice
test_img[:,:,1] = test_img_feat_slice
test_img[:,:,2] = test_img_feat_slice

plt.imshow(test_img[:,:,0], cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
directory_predicted_3 = os.path.join("..","data","test_images","3_Y_predicted.jpg")
directory_predicted_29 = os.path.join("..","data","test_images","29_Y_predicted.jpg")
directory_predicted_39 = os.path.join("..","data","test_images","20_Y_predicted.jpg")
directory_predicted_49 = os.path.join("..","data","test_images","31_Y_predicted.jpg")
directory_y = os.path.join("..","data","test_images","Y_truth.jpg")
directory_X = os.path.join("..","data","test_images","X_input.jpg")

visualize(
    input_image=cv2.imread(directory_X,0).squeeze(),
    ground_truth=cv2.imread(directory_y,0).squeeze(),
    predicted_3=cv2.imread(directory_predicted_3,0),
    predicted_29=cv2.imread(directory_predicted_29,0),
    predicted_39=cv2.imread(directory_predicted_39,0),
    predicted_49=cv2.imread(directory_predicted_49,0),### if putting the whole image, black and white, could it be the reason of the problem??
    
)


In [ ]:
import pandas as pd
pd.read_csv('history.csv')

____